# Advanced features

In [1]:
# Add BipartitePandas to system path, do not run this
# import sys
# sys.path.append('../../..')

## Import the BipartitePandas package

Make sure to install it using `pip install bipartitepandas`.

In [2]:
import bipartitepandas as bpd

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Get your data ready

For this notebook, we simulate data.

In [3]:
df = bpd.SimBipartite().simulate()
bdf = bpd.BipartiteDataFrame(i=df['i'], j=df['j'], y=df['y'], t=df['t'])
display(bdf)

,i,j,y,t
0,0,20,-2.187347,0
1,0,56,-1.044611,1
2,0,56,-1.593121,2
3,0,56,-1.627238,3
4,0,56,-2.765731,4
...,...,...,...,...
49995,9999,150,1.273217,0
49996,9999,108,-1.219111,1
49997,9999,169,0.790651,2
49998,9999,169,1.646778,3


## Advanced data cleaning

*Hint:* want details on all cleaning parameters? Run `bpd.clean_params().describe_all()`, or search through `bpd.clean_params().keys()` for a particular key, and then run `bpd.clean_params().describe(key)`.

#### Compute the largest connected component

Use the parameter `connectedness` to set the connectedness measure to use, and `component_size_variable` to choose how to measure component size (e.g. `firms` chooses the connected component with the greatest number of unique firm ids, while `workers` chooses the connected component with the greatest number of unique worker ids).

*Note:* connectedness is NOT NECESSARILY maintained between non-collapsed and collapsed formats. Therefore, if you plan to use connected, collapsed data, it is recommended to first clean the non-collapsed data with `connectedness=None`, next collapse the data, and finally clean the collapsed data with the connectedness measure you plan to use.

#### Set how to handle worker-year duplicates

Use the parameter `i_t_how` to customize how worker-year duplicates are handled.

#### Collapse at the match-level

If you drop the `t` column, collapsing will automatically collapse at the match level. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Avoid unnecessary copies

If you are working with a large dataset, you will want to avoid copies whenever possible. So set `copy=False`.

#### Avoid unnecessary sorts

If you know your data is sorted by `i` and `t` (or, if you aren't including a `t` column, just by `i`), then set `is_sorted=True`.

#### Avoid complicated loops

Sometimes workers leave a firm, then return to it (we call these workers *returners*). Returners can cause computational difficulties because sometimes intermediate firms get dropped (e.g. a worker goes from firm $A \to B \to A$, but firm $B$ gets dropped). This turns returners into stayers. This can change the largest connected set of firms, and if data is in collapsed format, requires the data to be recollapsed.

Because of these potential complications, if there are returners, many methods require loops that run until convergence.

These difficulties can be avoided by setting the parameter `drop_returns` (there are multiple ways to handle returners, they can be seen by running `bpd.clean_params().describe('drop_returns')`).

*Alternative:* another way to handle returners is to drop the `t` column. Then, sorting will automatically sort by `i` and `j`, which eliminates the possibility of returners. However, this prevents conversion to event study format (this can be bypassed with the `.construct_artificial_time()` method, but the data will likely have a meaningless order, rendering the event study uninterpretable).

#### Disable logging

Logging can slow down data cleaning. Set the parameter `log=False` when constructing your dataframe to turn off logging.

#### Install Intel(R) Extension for Scikit-learn

Intel(R) Extension for Scikit-learn ([GitHub](https://github.com/intel/scikit-learn-intelex)) can speed up KMeans clustering.

## Simpler constructor

If the columns in your Pandas dataframe are already named correctly, you can simply put the dataframe as a parameter into the BipartitePandas dataframe constructor.

In [4]:
bdf = bpd.BipartiteDataFrame(df).clean()
display(bdf)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,20,-2.187347,0,1,-0.967422,0,0,-1.335178
1,0,56,-1.044611,1,1,-0.967422,2,0,-0.604585
2,0,56,-1.593121,2,0,-0.967422,2,0,-0.604585
3,0,56,-1.627238,3,0,-0.967422,2,0,-0.604585
4,0,56,-2.765731,4,0,-0.967422,2,0,-0.604585
...,...,...,...,...,...,...,...,...,...
49995,9999,150,1.273217,0,1,0.000000,7,2,0.604585
49996,9999,108,-1.219111,1,2,0.000000,5,2,0.114185
49997,9999,169,0.790651,2,1,0.000000,8,2,0.908458
49998,9999,169,1.646778,3,1,0.000000,8,2,0.908458


## Restoring original ids

To restore original ids, we need to make sure the dataframe is tracking ids as they change.

We make sure the dataframe tracks ids as they change by setting `include_id_reference_dict=True`.

Notice that in this example we use `j / 2`, so that `j` will be modified during data cleaning.

In [5]:
bdf_original_ids = bpd.BipartiteDataFrame(i=df['i'], j=(df['j'] / 2), y=df['y'], t=df['t'], include_id_reference_dict=True).clean()
display(bdf_original_ids)

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m
0,0,0,-2.187347,0,1
1,0,1,-1.044611,1,1
2,0,1,-1.593121,2,0
3,0,1,-1.627238,3,0
4,0,1,-2.765731,4,0
...,...,...,...,...,...
49995,9999,63,1.273217,0,1
49996,9999,93,-1.219111,1,2
49997,9999,113,0.790651,2,1
49998,9999,113,1.646778,3,1


#### Merging in original ids

The method `.original_ids()` will return a dataframe that merges in the original ids.

In [6]:
bdf_original_ids.original_ids()

,i,j,y,t,m,original_j
0,0,0,-2.187347,0,1,10.0
1,0,1,-1.044611,1,1,28.0
2,0,1,-1.593121,2,0,28.0
3,0,1,-1.627238,3,0,28.0
4,0,1,-2.765731,4,0,28.0
...,...,...,...,...,...,...
49995,9999,63,1.273217,0,1,75.0
49996,9999,93,-1.219111,1,2,54.0
49997,9999,113,0.790651,2,1,84.5
49998,9999,113,1.646778,3,1,84.5


## Filling in missing years as unemployed

The method `.fill_periods()` (for *Long* format) will fill in rows for missing intermediate periods. Note that this method will not work with custom columns.

In this example, we drop periods 1-3, then fill them in:

In [7]:
bdf_missing = bdf[(bdf['t'] == 0) | (bdf['t'] == 4)].clean()
display(bdf_missing.fill_periods())

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how=False)
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


,i,j,y,t,m,alpha,k,l,psi
0,0,20,-2.187347,0,1,-0.967422,0.0,0.0,-1.335178
1,0,-1,NaN,1,NaN,NaN,NaN,NaN,NaN
2,0,-1,NaN,2,NaN,NaN,NaN,NaN,NaN
3,0,-1,NaN,3,NaN,NaN,NaN,NaN,NaN
4,0,56,-2.765731,4,1,-0.967422,2.0,0.0,-0.604585
...,...,...,...,...,...,...,...,...,...
49995,9999,150,1.273217,0,1,0.000000,7.0,2.0,0.604585
49996,9999,-1,NaN,1,NaN,NaN,NaN,NaN,NaN
49997,9999,-1,NaN,2,NaN,NaN,NaN,NaN,NaN
49998,9999,-1,NaN,3,NaN,NaN,NaN,NaN,NaN


## Getting extended event study dataframes

BipartitePandas allows you to use *Long* format data to generate event studies with more than 2 periods.

You can specify:

- what column signals a transition (e.g. if `j` is used, a transition is when a worker moves firms)
- which column(s) should be treated as the event study outcome
- how many periods before and after the transition should be considered
- whether the pre- and/or post-trends must be stable, and for which column(s)

We consider an example where `j` is the transition column, `y` is the outcome column, and with pre- and post-trends of length 2 that are required to be at the same firm. Note that `y_f1` is the first observation after the individual moves firms.

In [8]:
es_extended = bdf.get_extended_eventstudy(transition_col='j', outcomes='y', periods_pre=2, periods_post=2, stable_pre='j', stable_post='j')
display(es_extended)

,i,t,y_l2,y_l1,y_f1,y_f2
0,1,3,0.364982,-0.290097,-2.620315,0.070954
1,22,2,-0.138161,0.082168,-0.424210,-0.617168
2,23,3,0.591211,3.505691,2.440164,2.870888
3,24,3,0.693831,-1.171445,-0.963292,-1.396924
4,25,3,-3.543229,-1.446526,-1.306608,-0.289481
...,...,...,...,...,...,...
2458,9982,3,1.903089,1.736533,0.112897,0.281145
2459,9983,2,-0.563675,0.800055,0.713616,-1.775842
2460,9985,2,1.567863,1.639899,1.209529,0.231509
2461,9989,2,-1.323139,-2.257726,-0.996262,-0.882585
